In [1]:
import random_VQE_data_prep
import pickle
import pandas as pd
import re
import numpy as np
from sklearn import linear_model
from sklearn import metrics
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [4]:
random_VQE_data_prep.data_storage(500)

In [5]:
file = open("./pickles/circ_0.pickle", "rb")
pickle.load(file)

defaultdict(None,
            {'Quantum_circuit': [<qiskit.circuit.quantumcircuit.QuantumCircuit at 0x1427ecbd0>],
             'num_layers': 4,
             'obervable': 'IIZII',
             'ideal_expectation': -0.13528096997928452,
             'noisy_expectation': 0.11802})

In [ ]:
def count_gate_paris(df, gates:str):
    l = df[gates]
    res = dict()
    for i in range(len(l)):
        tmp = str(l[i])
        if tmp in res.keys():
            res[tmp] += 1
        else:
            res[tmp] = 1
    return res

In [ ]:
def extract_features(path):
    file = open(path, "rb")
    dict_ = pickle.load(file)
    #print(dict)
    #print(len(dict["Quantum_circuit"]))
    data = dict_["Quantum_circuit"][0].__dict__["_data"]
    #print(dict["Quantum_circuit"][0].draw())
    

    qubits = str(data)
    #print(qubits)

    # Split input_string before each "CircuitInstruction"
    instructions = qubits.split("CircuitInstruction")

    # Remove the empty string at the beginning (resulting from the initial split)
    instructions = instructions[1:]

    #patterns to get different values from datastring
    pattern_nq = r"num_qubits=(\d+)"
    pattern_nc = r"num_clbits=(\d+)"
    pattern_n = r"name='(\w+)'"
    pattern_p = r"params=\[(.*?)\]"
    pattern_gates = r"Qubit\(QuantumRegister\(5, 'q'\), (\d+)\)"
    #pattern_clb = r'clbits=\(\)\)' -> possibly clbits relevant?

    # Extracted numbers
    # find all values and store them in array to build df
    numbers = [int(match) for match in re.findall(pattern_nq, qubits)]
    clbits = [int(match_cl) for match_cl in re.findall(pattern_nq, qubits)]
    name = [match_name for match_name in re.findall(pattern_n, qubits)]
    params = [match_p for match_p in re.findall(pattern_p, qubits)]

    # loop over all Instructions to get per instruction a list of used qubits
    gates_all = []
    for instr in enumerate(instructions):
        gates = [match_gates for match_gates in re.findall(pattern_gates, str(instr))]
        gates_all.append(gates)


    #create new df to show parameters
    df_new =pd.DataFrame({'name': name, 'num_qubits': numbers, 'num_clbits': clbits, 'params': params, 'gates': gates_all})
    df_new = df_new.sort_values(by='num_qubits', ascending=False)
    df_new = df_new[df_new['num_qubits'] == 2].reset_index()

    res = dict()
    res['noisy_expectation'] = dict_['noisy_expectation']
    res['num_layers'] = dict_['num_layers']
    #res['observable'] = dict_['obervable']
    tmp = count_gate_paris(df_new, 'gates')
    for i in tmp.keys():
        res['count_' + i] = tmp[i]
    res['N2QG'] = len(df_new)
    res['target'] = dict_['ideal_expectation']

    return res 

In [ ]:
# quick and dirty test for r2-score without using observable
a = list()
for i in range(1000):
    a.append(extract_features(f"./pickles/circ_{i}.pickle"))
df = pd.DataFrame(a)
display(df)

train_x = df[:900]['noisy_expectation'].to_numpy().reshape(-1, 1)
train_y = df[:900]['target']

test_x = df[900:]['noisy_expectation'].to_numpy().reshape(-1, 1)
test_y = df[900:]['target']

alpha = 0
model = linear_model.Ridge(alpha=alpha)
model = model.fit(train_x, train_y)
print('Ridge with alpha = ' + str(alpha) + ': ' + str(metrics.r2_score(test_y, model.predict(test_x))))

model = linear_model.LinearRegression()
model = model.fit(train_x, train_y)
print('OLS: ' + str(metrics.r2_score(test_y, model.predict(test_x))))

tsne = TSNE(n_components=1)
new_data = tsne.fit_transform(df.drop(columns=['target']))

fig = plt.figure()
ax = fig.add_subplot()#projection='3d')
ax.scatter(df['noisy_expectation'], df['target'])#[new_data[i][1] for i in range(len(new_data))], df['target'])
ax.plot(test_x, model.predict(test_x), color='red')
plt.show()

TypeError: CircuitInstruction.__new__() missing 1 required positional argument: 'operation'

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Fixing random state for reproducibility
np.random.seed(19680801)


def randrange(n, vmin, vmax):
    """
    Helper function to make an array of random numbers having shape (n, )
    with each number distributed Uniform(vmin, vmax).
    """
    return (vmax - vmin)*np.random.rand(n) + vmin

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

n = 100

# For each set of style and range settings, plot n random points in the box
# defined by x in [23, 32], y in [0, 100], z in [zlow, zhigh].
for m, zlow, zhigh in [('o', -50, -25), ('^', -30, -5)]:
    xs = randrange(n, 23, 32)
    ys = randrange(n, 0, 100)
    zs = randrange(n, zlow, zhigh)
    ax.scatter(xs, ys, zs, marker=m)

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()

: 

: 